## Wikipedia Article Retrieval Tool

This section defines a simple Python function, `get_article`, that uses the `wikipedia` Python package to search for and retrieve the content of a Wikipedia article based on a search term. This function will be used as a tool for the agent to fetch up-to-date information from Wikipedia.

In [1]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

## Example: Fetching and Previewing Wikipedia Articles

Here, we demonstrate how to use the `get_article` function to retrieve and preview the content of Wikipedia articles for various search terms, such as "Avengers: Doomsday", "Nezha 2", "History of Malaysia", and "Iron Man". Only a preview of the article content is printed for brevity.

In [2]:
article = get_article("Avengers: Doomsday")
print(article[:1000]) # article is very long, so let's just print a preview

Avengers: Doomsday is an upcoming American superhero film based on the Marvel Comics superhero team the Avengers. Produced by Marvel Studios and AGBO, and distributed by Walt Disney Studios Motion Pictures, it is intended to be the fifth installment in the Avengers film series following Avengers: Endgame (2019) and the 39th film in the Marvel Cinematic Universe (MCU). Directed by Anthony and Joe Russo and written by Michael Waldron and Stephen McFeely, the film features an ensemble cast including Chris Hemsworth, Vanessa Kirby, Anthony Mackie, Sebastian Stan, Letitia Wright, Paul Rudd, Wyatt Russell, Tenoch Huerta Mejía, Ebon Moss-Bachrach, Simu Liu, Florence Pugh, Kelsey Grammer, Lewis Pullman, Danny Ramirez, Joseph Quinn, David Harbour, Winston Duke, Hannah John-Kamen, Tom Hiddleston, Patrick Stewart, Ian McKellen, Alan Cumming, Rebecca Romijn, James Marsden, Channing Tatum, Pedro Pascal, and Robert Downey Jr. In the film, the Avengers, Wakandans, Fantastic Four, New Avengers, and th

In [3]:
article = get_article("Nezha 2")
print(article[:500]) # article is very long, so let's just print a preview

Ne Zha 2 (Chinese: 哪吒之魔童闹海; pinyin: Nézhā zhī Mótóng nào hǎi; also known as 哪吒2; Nézhā èr) is a 2025 Chinese animated fantasy adventure comedy film written and directed by Jiaozi. The direct sequel to Ne Zha (2019), it is based on the Chinese mythological character and Xu Zhonglin's 16th-century novel Investiture of the Gods (Fengshen Yanyi). The film takes up the story of Chinese mythological character Ne Zha and his friend Ao Bing. After a sacrifice, only Ne Zha’s body can be recreated, althou


In [4]:
article = get_article("History of Malaysia")
print(article[:3000]) #article is super long so let's just print a preview

Malaysia is a modern concept, created in the second half of the 20th century. However, contemporary Malaysia regards the entire history of Malaya and Borneo, spanning thousands of years back to prehistoric times, as its own history. Significant events in Malaysia's modern history include the formation of the federation, the separation of Singapore, the racial riots, Mahathir Mohamad's era of industrialisation and privatisation, and the nation's political upheavals of the late 20th and early 21st centuries.
The first evidence of archaic human occupation in the region dates back at least 1.83 million years, while the earliest remnants of anatomically modern humans are approximately 40,000 years old. The ancestors of the present-day population of Malaysia entered the area in multiple waves during prehistoric and historical times.
Hinduism and Buddhism from India and China dominated early regional history, reaching their peak from the 7th to the 13th centuries during the reign of the Sumat

In [5]:
article = get_article("Iron Man")
print(article[:1000]) #article is super long so let's just print a preview

Iron Man is a superhero appearing in American comic books published by Marvel Comics. Co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby, the character first appeared in Tales of Suspense #39 in 1962 (cover dated March 1963) and received his own title with Iron Man #1 in 1968. Shortly after his creation, Iron Man became a founding member of the superhero team, the Avengers, alongside Thor, Ant-Man, the Wasp, and the Hulk. Iron Man stories, individually and with the Avengers, have been published consistently since the character's creation.
Iron Man is the superhero persona of Anthony Edward "Tony" Stark, a businessman and engineer who runs the weapons manufacturing company Stark Industries. When Stark was captured in a war zone and sustained a severe heart wound, he built his Iron Man armor and escaped his captors. Iron Man's suits of armor grant him superhuman strength, flight, energy projection, and other abili

## Tool Schema Definition for Agent Use

This cell defines a tool schema dictionary, `article_search_tool`, which describes the Wikipedia retrieval tool in a format compatible with agentic LLM APIs (like Anthropic Claude). The schema includes the tool's name, description, input schema, and required fields.

In [6]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

## Setting Up Anthropic Client and Making a Tool-Use Request

This section initializes the Anthropic API client and demonstrates how to send a message to the Claude model, asking a question that may require tool use (e.g., "What is the box office for Nezha 2?"). The tool schema is provided to the model, enabling it to call the Wikipedia tool if needed.

In [7]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [{"role": "user", "content": "What is the box office for Nezha 2"}]

response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )

In [8]:
response.content

[TextBlock(text='I\'ll help you find information about the box office performance for "Nezha 2." Let me retrieve that information from Wikipedia.', type='text'),
 ToolUseBlock(id='toolu_012ZdbPs9sC6bPqMVfr1pd1f', input={'search_term': 'Nezha 2'}, name='get_article', type='tool_use')]

## Appending the Assistant's Tool Use to the Conversation

This cell appends the assistant's tool use response to the ongoing conversation history, preparing for the next step in the agentic interaction.

In [9]:
messages.append({"role": "assistant", "content": response.content})

## Extracting Tool Use Information

This section extracts the tool name and input parameters from the model's tool use response, so that the tool can be called programmatically.

In [10]:
# This is a simple, but brittle way of getting the tool use information
# We're simply taking the last block from Claude's response.
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input
print("Tool name: ", tool_name)
print("Tool input", tool_input)

Tool name:  get_article
Tool input {'search_term': 'Nezha 2'}


## Executing the Tool and Returning Results

Here, the code checks if the model requested the Wikipedia tool, executes the tool with the provided input, and prints a preview of the Wikipedia article content.

In [11]:
if tool_name == "get_article":
    search_term = tool_input["search_term"]
    wiki_result = get_article(search_term)
    print(f"Searching Wikipedia for {search_term}")
    print("WIKIPEDIA PAGE CONTENT:")
    print(wiki_result[:500]) #just printing a small bit of the article because it's so long

Searching Wikipedia for Nezha 2
WIKIPEDIA PAGE CONTENT:
Ne Zha 2 (Chinese: 哪吒之魔童闹海; pinyin: Nézhā zhī Mótóng nào hǎi; also known as 哪吒2; Nézhā èr) is a 2025 Chinese animated fantasy adventure comedy film written and directed by Jiaozi. The direct sequel to Ne Zha (2019), it is based on the Chinese mythological character and Xu Zhonglin's 16th-century novel Investiture of the Gods (Fengshen Yanyi). The film takes up the story of Chinese mythological character Ne Zha and his friend Ao Bing. After a sacrifice, only Ne Zha’s body can be recreated, althou


## Constructing a Tool Result Message

This cell shows how to construct a message containing the tool result, formatted according to the agentic API's expectations, so it can be sent back to the model.

In [12]:
{
  "role": "user",
  "content": [
    {
      "type": "tool_result",
      "tool_use_id": "toolu_01A09q90qw90lq917835lq9",
      "content": "The result of actually calling the tool goes here"
    }
  ]
}

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_01A09q90qw90lq917835lq9',
   'content': 'The result of actually calling the tool goes here'}]}

## Sending the Tool Result to the Conversation

The tool result message is appended to the conversation history, allowing the agent to use the tool's output in its next response.

In [13]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": wiki_result
        }
    ]
}

In [14]:
tool_response

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_012ZdbPs9sC6bPqMVfr1pd1f',
   'content': 'Ne Zha 2 (Chinese: 哪吒之魔童闹海; pinyin: Nézhā zhī Mótóng nào hǎi; also known as 哪吒2; Nézhā èr) is a 2025 Chinese animated fantasy adventure comedy film written and directed by Jiaozi. The direct sequel to Ne Zha (2019), it is based on the Chinese mythological character and Xu Zhonglin\'s 16th-century novel Investiture of the Gods (Fengshen Yanyi). The film takes up the story of Chinese mythological character Ne Zha and his friend Ao Bing. After a sacrifice, only Ne Zha’s body can be recreated, although he carries Ao Bing’s spirit within. Ne Zha calls on this spirit in his fight against wicked Master Shen.\nNe Zha 2 was released in theaters across China on 29 January 2025, coinciding with the first day of the Chinese New Year. Like its predecessor, the film received highly positive reviews from critics for its animation, action sequences, worldbuilding, and emotional core

In [15]:
messages.append(tool_response)

In [16]:
follow_up_response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

In [17]:
follow_up_response.content[0].text

'Based on the Wikipedia article, I can provide you with information about the box office performance of "Ne Zha 2" (also known as "Ne Zha: The demon child churns the sea"):\n\nAccording to the article, "Ne Zha 2" has been an extraordinary box office success with a current gross of more than $2.2 billion against a production budget of US$80 million. The film has broken numerous box office records both inside and outside China:\n\nKey box office achievements:\n- It became the highest-grossing film in a single box office territory\n- Globally became the highest-grossing animated film\n- Became the highest-grossing non-English language film (being the first to gross over $2 billion)\n- The first animated film in history to cross the $2 billion mark\n- Ranks as the highest-grossing film of 2025\n- The fifth-highest-grossing film of all time\n- The highest-selling animated film in terms of ticket sales\n\nIn China specifically:\n- Reached 1.1 billion yuan (US$137 million) in just three days\

## Model Follow-up: Final Answer Generation

This section sends the updated conversation (including the tool result) back to the model, prompting it to generate a final answer that incorporates the information retrieved from Wikipedia.

In [18]:

def answer_question(question):
    messages = [{"role": "user", "content": question}]

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )
    
    if(response.stop_reason == "tool_use"):
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": response.content})

        if tool_name == "get_article":
            search_term = tool_input["search_term"]
            print(f"Claude wants to get an article for {search_term}")
            wiki_result = get_article(search_term) #get wikipedia article content
            #construct our tool_result message
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": wiki_result
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                messages=messages,
                max_tokens=1000,
                tools=[article_search_tool]
            )
            print("Claude's final answer:")
            print(response.content[0].text)

    else:
        print("Claude did not call our tool")
        print(response.content[0].text)

## Agentic Tool Use: Full Question-Answering Loop

This cell defines a reusable function, `answer_question`, that demonstrates the full agentic tool-use loop: sending a question to the model, detecting tool use, executing the tool, sending the result, and printing the model's final answer.

In [19]:
answer_question("What are the names of all the Avengers films in the Marvel Cinematic Universe?")

Claude wants to get an article for Avengers (Marvel Cinematic Universe)
Claude's final answer:
Based on the Wikipedia article, here are all the Avengers films in the Marvel Cinematic Universe:

1. **The Avengers** (2012) - The first Avengers film that brought together the original six heroes: Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye.

2. **Avengers: Age of Ultron** (2015) - The second film featuring the team facing the artificial intelligence Ultron.

3. **Avengers: Infinity War** (2018) - The third film where the Avengers and their allies confront Thanos who seeks to collect all six Infinity Stones.

4. **Avengers: Endgame** (2019) - The fourth film that concludes the Infinity Saga and depicts the final battle against Thanos.

Additionally, two future Avengers films have been announced:

5. **Avengers: Doomsday** (planned for 2026)
6. **Avengers: Secret Wars** (planned for 2027)

Both of these upcoming films will be part of the MCU's Phase Six and will conclude 

## Web Search with Anthropic Claude API

This code demonstrates how to use Anthropic's Claude API to perform web searches using the built-in web_search_20250305 tool. This allows Claude to search the web for current information and provide up-to-date responses.

### Overview
The code creates a simple web search query using Claude's web search capability,which is particularly useful for getting current information that might not be available in the model's training data.

### Key Components

### Imports
- `anthropic`: The official Anthropic Python client
- `IPython.display`: For enhanced display formatting in Jupyter notebooks
- `json`: For pretty-printing response objects

### API Configuration
- Model: `claude-opus-4-1-20250805` (latest Claude Opus model)
- Max tokens: 1024 (controls response length)
- Tool: `web_search_20250305` with max 5 uses per request

### Usage

1. **Setup**: Ensure you have the Anthropic API key configured
2. **Query**: Modify the user message to ask any question requiring current information
3. **Execute**: Run the code to get Claude's response with web search results

### Example Use Cases
* Getting current software version information
* Finding recent news or updates
* Researching current events
* Checking latest technology trends
* Verifying current facts or statistics

### Response Format
The response will contain:
* Claude's analysis of the search results
* Relevant information found through web search
* Citations to source URLs when available

### Cost Considerations
* Web search tool usage incurs additional costs
* Each search counts toward your API usage quota
* Monitor usage in your Anthropic dashboard


In [20]:
import anthropic
from IPython.display import display, Markdown
import json 

client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-opus-4-1-20250805",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Can you teach me how to code an AI agent using the Anthropic API?"
        }
    ],
    tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 5
    }]
)
print(response)

Message(id='msg_01LJBDLsVKq5HcnQUciheDXp', content=[TextBlock(text='I can help you learn how to code an AI agent using the Anthropic API! Let me search for the most current information about their API documentation and implementation details.', type='text'), TextBlock(text=None, type='server_tool_use', id='srvtoolu_01QBWewJVesccz4d3Te7sNa9', name='web_search', input={'query': 'Anthropic API documentation Claude'}), TextBlock(text=None, type='web_search_tool_result', tool_use_id='srvtoolu_01QBWewJVesccz4d3Te7sNa9', content=[{'type': 'web_search_result', 'title': 'Anthropic API', 'url': 'https://docs.anthropic.com/en/home', 'encrypted_content': 'EugFCioIBxgCIiRjODRlYmM3Ny01ZjQ3LTRiYTItYTJmOC1lODlkOTljNWRmM2ESDCjrUFGl61slAG6/WRoMrkAHWYu4KT8I3hf5IjBR5t3Ue9kY81/XVMdXQ8PpETguwCpvpgGtSFM418gi+bJc3oYna3Zlx2yaPxChQB8q6wQ1q/l3MtjbvqTgkOnKiusW4I0FwDvqUNy/ypB0RY764R40wfOcFZdz03lH8OfBa6IE6pL8ddZUiJ7GPREft2gMkHIZHoMWKq6jTCLHicp5Tt2mK5XOX58auL72uJNURNliJ1F+NZGLPt2/UQ81fjpPDm2TUxx9s8yUhC2pS/yZCLA8wHNU

In [21]:
# Option 1: Pretty print the entire response as JSON
print("=== Full Response ===")
print(json.dumps(response.model_dump(), indent=2))

# Option 2: Display just the content in a formatted way
print("\n=== Response Content ===")
for i, content_block in enumerate(response.content):
    print(f"\n--- Content Block {i+1} ---")
    print(f"Type: {content_block.type}")
    if hasattr(content_block, 'text'):
        print(f"Text: {content_block.text}")
    if hasattr(content_block, 'name'):
        print(f"Tool Name: {content_block.name}")
        print(f"Tool Input: {content_block.input}")

# Option 3: Display as markdown for better formatting
print("\n=== Formatted Display ===")
formatted_output = "## Claude Response\n\n"
for i, content_block in enumerate(response.content):
    formatted_output += f"### Block {i+1}\n"
    formatted_output += f"**Type:** {content_block.type}\n\n"
    
    if hasattr(content_block, 'text'):
        formatted_output += f"**Content:**\n{content_block.text}\n\n"
    if hasattr(content_block, 'name'):
        formatted_output += f"**Tool:** {content_block.name}\n"
        formatted_output += f"**Input:** {content_block.input}\n\n"

display(Markdown(formatted_output))

=== Full Response ===
{
  "id": "msg_01LJBDLsVKq5HcnQUciheDXp",
  "content": [
    {
      "text": "I can help you learn how to code an AI agent using the Anthropic API! Let me search for the most current information about their API documentation and implementation details.",
      "type": "text"
    },
    {
      "text": null,
      "type": "server_tool_use",
      "id": "srvtoolu_01QBWewJVesccz4d3Te7sNa9",
      "name": "web_search",
      "input": {
        "query": "Anthropic API documentation Claude"
      }
    },
    {
      "text": null,
      "type": "web_search_tool_result",
      "tool_use_id": "srvtoolu_01QBWewJVesccz4d3Te7sNa9",
      "content": [
        {
          "type": "web_search_result",
          "title": "Anthropic API",
          "url": "https://docs.anthropic.com/en/home",
          "encrypted_content": "EugFCioIBxgCIiRjODRlYmM3Ny01ZjQ3LTRiYTItYTJmOC1lODlkOTljNWRmM2ESDCjrUFGl61slAG6/WRoMrkAHWYu4KT8I3hf5IjBR5t3Ue9kY81/XVMdXQ8PpETguwCpvpgGtSFM418gi+bJc3oYna3Zlx2

/opt/anaconda3/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `literal['text']` - serialized value may not be as expected [input_value='server_tool_use', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `ToolUseBlock` - serialized value may not be as expected [input_value=TextBlock(text=None, type... documentation Claude'}), input_type=TextBlock])
  PydanticSerializationUnexpectedValue(Expected `literal['text']` - serialized value may not be as expected [input_value='web_search_tool_result', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `ToolUseBlock` - serialized value may not be as expected [input_value=TextBlock(text=None, type...gD', 'page_age': None}]), input_type=TextBlock])
  PydanticSerializationUnexpectedValue(Expected `literal['text']` - serialized value may not be as expected [input_value='server_tool_use', input_type=str])
  PydanticSerialization

## Claude Response

### Block 1
**Type:** text

**Content:**
I can help you learn how to code an AI agent using the Anthropic API! Let me search for the most current information about their API documentation and implementation details.

### Block 2
**Type:** server_tool_use

**Content:**
None

**Tool:** web_search
**Input:** {'query': 'Anthropic API documentation Claude'}

### Block 3
**Type:** web_search_tool_result

**Content:**
None

### Block 4
**Type:** server_tool_use

**Content:**
None

**Tool:** web_search
**Input:** {'query': 'Anthropic Claude API Python tutorial example'}

### Block 5
**Type:** web_search_tool_result

**Content:**
None

### Block 6
**Type:** text

**Content:**
Great! I'll teach you how to code an AI agent using the Anthropic API. Let me provide you with a comprehensive guide based on the latest documentation.

## Getting Started with the Anthropic API

### Step 1: Set Up Your Account and Get API Key



### Block 7
**Type:** text

**Content:**
First, visit the Anthropic Console and create an account. You'll need to provide basic information and verify your email. Once your account is set up, navigate to Get API keys section and click Create Key to generate your API key.

### Block 8
**Type:** text

**Content:**


### Step 2: Install the Python SDK



### Block 9
**Type:** text

**Content:**
The Anthropic Python library provides convenient access to the Anthropic REST API from any Python 3.8+ application. It includes type definitions for all request params and response fields, and offers both synchronous and asynchronous clients.

### Block 10
**Type:** text

**Content:**


Install the SDK using pip:
```bash
pip install anthropic
```

### Step 3: Set Up Your Environment



### Block 11
**Type:** text

**Content:**
We recommend using python-dotenv to add ANTHROPIC_API_KEY="my-anthropic-api-key" to your .env file so that your API Key is not stored in source control.

### Block 12
**Type:** text

**Content:**


You can also set it as an environment variable:
```bash
export ANTHROPIC_API_KEY='your-api-key-here'
```

### Step 4: Basic AI Agent Code

Here's a simple example to get you started:

```python
import os
from anthropic import Anthropic

# Initialize the client
client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),  # This is the default and can be omitted
)

# Create a message
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude! Can you help me understand Python?"
        }
    ]
)

print(message.content)
```

### Step 5: Building a More Advanced AI Agent

Here's an example of a more sophisticated AI agent with system prompts and conversation handling:

```python
import anthropic

class AIAgent:
    def __init__(self, system_prompt="You are a helpful AI assistant."):
        self.client = anthropic.Anthropic()
        self.system_prompt = system_prompt
        self.conversation_history = []
    
    def chat(self, user_input):
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_input
        })
        
        # Create message with system prompt
        response = self.client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1000,
            temperature=0.7,
            system=self.system_prompt,
            messages=self.conversation_history
        )
        
        # Add assistant response to history
        assistant_message = {
            "role": "assistant",
            "content": response.content[0].text
        }
        self.conversation_history.append(assistant_message)
        
        return response.content[0].text
    
    def reset_conversation(self):
        self.conversation_history = []

# Usage example
agent = AIAgent(system_prompt="You are a Python programming expert. Help users write clean, efficient code.")

# Have a conversation
response1 = agent.chat("How do I read a file in Python?")
print(response1)

response2 = agent.chat("Can you show me how to handle errors when doing that?")
print(response2)
```

### Step 6: Using Tool Capabilities



### Block 13
**Type:** text

**Content:**
Anthropic released official support for tool use through their API, allowing you to extend Claude's capabilities with custom functions.

### Block 14
**Type:** text

**Content:**


Here's an example with tool usage:

```python
import anthropic

def get_weather(location):
    # Mock function - replace with actual API call
    return f"The weather in {location} is sunny and 72°F"

client = anthropic.Anthropic()

tools = [
    

